In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import utils as np_utils
from keras.utils import to_categorical
print('Using TensorFlow version', tf.__version__)

Using TensorFlow version 2.3.0


In [ ]:
import pandas as pd 
import io

url_train = 'https://raw.githubusercontent.com/bondadakumarsai/DeeplearningbasedIQClassification/master/IQ_samples/trainDataLabels.csv'
url_test = 'https://raw.githubusercontent.com/bondadakumarsai/DeeplearningbasedIQClassification/master/IQ_samples/testData.csv'
df_train = pd.read_csv(url_train)
df_test  = pd.read_csv(url_test)

df_train['Input'] = df_train['Input'].str.replace('i','j').apply(lambda x: np.complex(x))
#df['Label'] = df['Label']
data = df_train.values
X_train = data[:,1]
X_train = X_train.reshape(len(X_train),1)
Y_train = data[:,2].real.reshape(len(X_train),1)
print(X_train[0:5])
print(Y_train[0:5])
print(X_train.shape)
print(Y_train.shape)

df_test['Input'] = df_test['Input'].str.replace('i','j').apply(lambda x: np.complex(x))
#df['Label'] = df['Label']
data = df_test.values
X_test = data[:,1]
X_test = X_test.reshape(len(X_test),1)
print(X_test.shape)

[[-0.78145+0.65703j]
 [-0.35728+1.0268j ]
 [ 0.38244+0.32534j]
 [-0.95181-0.86482j]
 [ 0.95482+1.0778j ]]
[[1.]
 [2.]
 [2.]
 [2.]
 [3.]]
(20000, 1)
(20000, 1)
(10000, 1)


In [ ]:
y_train_encoded = to_categorical(Y_train)
print('y_train shape: ', y_train_encoded.shape)


y_train shape:  (20000, 4)


In [ ]:
y_train_encoded[:2]

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [ ]:
#x_train_reshaped = np.reshape(X_train, (20000, 1))
x = abs(X_train)
x_phase = np.angle(X_train)
x_input = np.concatenate((x, x_phase,x*x_phase, x**2, x_phase**2,(x *x_phase)**2 ,x**4, x_phase**4, x**6, x_phase**6),axis=1)

x = abs(X_test)
x_phase = np.angle(X_test)
x_input_test = np.concatenate((x, x_phase, x*x_phase, x**2, x_phase**2,(x *x_phase)**2 ,x**4, x_phase**4, x**6, x_phase**6),axis=1)

print('x: ', x_input)

x:  [[ 1.02095667e+00  2.44247384e+00  2.49365996e+00 ...  3.55893193e+01
   1.13251475e+00  2.12314435e+02]
 [ 1.08718317e+00  1.90564799e+00  2.07178842e+00 ...  1.31877506e+01
   1.65126326e+00  4.78912407e+01]
 [ 5.02102051e-01  7.04897719e-01  3.53930590e-01 ...  2.46890524e-01
   1.60233002e-02  1.22675159e-01]
 ...
 [ 1.36096803e+00  2.36468943e+00  3.21826671e+00 ...  3.12677360e+01
   6.35459010e+00  1.74841553e+02]
 [ 9.96447061e-01 -2.33123428e+00 -2.32295155e+00 ...  2.95354562e+01
   9.78870823e-01  1.60514963e+02]
 [ 8.10456060e-01  2.77833792e+00  2.25172080e+00 ...  5.95854554e+01
   2.83384991e-01  4.59949758e+02]]


In [ ]:
print(x_input)

[[ 1.02095667e+00  2.44247384e+00  2.49365996e+00 ...  3.55893193e+01
   1.13251475e+00  2.12314435e+02]
 [ 1.08718317e+00  1.90564799e+00  2.07178842e+00 ...  1.31877506e+01
   1.65126326e+00  4.78912407e+01]
 [ 5.02102051e-01  7.04897719e-01  3.53930590e-01 ...  2.46890524e-01
   1.60233002e-02  1.22675159e-01]
 ...
 [ 1.36096803e+00  2.36468943e+00  3.21826671e+00 ...  3.12677360e+01
   6.35459010e+00  1.74841553e+02]
 [ 9.96447061e-01 -2.33123428e+00 -2.32295155e+00 ...  2.95354562e+01
   9.78870823e-01  1.60514963e+02]
 [ 8.10456060e-01  2.77833792e+00  2.25172080e+00 ...  5.95854554e+01
   2.83384991e-01  4.59949758e+02]]


In [ ]:
x_mean = np.mean(x_input,axis=0)#reshape(x_input.shape[1],1)
x_mean = x_mean.reshape(1,x_input.shape[1])
x_std = np.std(x_input,axis =0)
x_std = x_std.reshape(1, x_input.shape[1])

x_mean_test = np.mean(x_input_test,axis=0)#reshape(x_input.shape[1],1)
x_mean_test = x_mean.reshape(1,x_input_test.shape[1])
x_std_test = np.std(x_input,axis =0)
x_std_test = x_std.reshape(1, x_input_test.shape[1])



print('mean: ', x_mean.shape)
print('std: ', x_std.shape)

mean:  (1, 10)
std:  (1, 10)


In [ ]:
epsilon = 1e-10
x_train_norm = (x_input - x_mean)/(x_std + epsilon)
x_test_norm = (x_input_test - x_mean_test)/(x_std_test + epsilon)



In [ ]:
print(set(x_train_norm[0]))

{0.19500166658276846, 1.3592900815397813, 1.3806497632962795, 1.0351476532964838, 0.896774883400021, -0.16398144827080052, 0.8822529935513699, -0.27344868811842354, 0.6418932990223964, 0.059812939300189195}


In [ ]:
from tensorflow.keras.initializers import GlorotNormal
initializer = tf.keras.initializers.GlorotNormal(seed=1)

from tensorflow.keras.optimizers.schedules import ExponentialDecay

#lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#    initial_learning_rate=1e-2,
#    decay_steps=10000,
#    decay_rate=0.9)
#optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout

model = Sequential([
    Dense(512, activation = 'relu', input_shape = (10,),kernel_initializer=initializer),
    Dense(512, activation = 'relu',kernel_initializer=initializer),
    Dense(512, activation = 'relu',kernel_initializer=initializer),
    Dense(512, activation = 'relu',kernel_initializer=initializer), 
    Dense(512, activation = 'relu',kernel_initializer=initializer),
    Dense(512, activation = 'relu',kernel_initializer=initializer), 
    Dense(512, activation = 'relu',kernel_initializer=initializer),
    Dense(512, activation = 'relu',kernel_initializer=initializer),
    Dense(512, activation = 'relu',kernel_initializer=initializer), 
    Dense(512, activation = 'relu',kernel_initializer=initializer),
    Dense(4, activation = 'softmax',kernel_initializer=initializer)
])

ValueError: ignored

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 512)               5632      
_________________________________________________________________
dense_42 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_43 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_44 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_45 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_46 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_47 (Dense)             (None, 512)              

In [ ]:
h = model.fit(
    x_train_norm,
    y_train_encoded,
    batch_size = 4096,
    epochs = 50
)

Epoch 1/50
5/5 [==============================] - 4s 729ms/step - loss: 1.2392 - accuracy: 0.3304
Epoch 2/50
5/5 [==============================] - 4s 728ms/step - loss: 1.0851 - accuracy: 0.3379
Epoch 3/50
5/5 [==============================] - 4s 729ms/step - loss: 0.9800 - accuracy: 0.4089
Epoch 4/50
5/5 [==============================] - 4s 731ms/step - loss: 0.8532 - accuracy: 0.5931
Epoch 5/50
5/5 [==============================] - 4s 729ms/step - loss: 0.7345 - accuracy: 0.6025
Epoch 6/50
5/5 [==============================] - 4s 728ms/step - loss: 0.7073 - accuracy: 0.6041
Epoch 7/50
5/5 [==============================] - 4s 729ms/step - loss: 0.6819 - accuracy: 0.6081
Epoch 8/50
5/5 [==============================] - 4s 731ms/step - loss: 0.6501 - accuracy: 0.6271
Epoch 9/50
5/5 [==============================] - 4s 728ms/step - loss: 0.5922 - accuracy: 0.6392
Epoch 10/50
5/5 [==============================] - 4s 728ms/step - loss: 0.5872 - accuracy: 0.6615
Epoch 11/50
5/5 [==

In [ ]:
loss, accuracy = model.evaluate(x_test_norm)

print('test set accuracy: ', accuracy * 100)

313/313 [==============================] - 0s 612us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
test set accuracy:  0.0


In [ ]:
preds = model.predict(x_test_norm)
pred = np.argmax(preds,axis = 1).reshape(10000,1)
id = np.arange(1,10001).reshape(10000,1)
prediction = np.concatenate((id, pred),axis=1)

import csv   
      
# field names   
fields = ['Id', 'Predicted Label']  

filename = "submission.csv"
      
# writing to csv file   
with open(filename, 'w') as csvfile:   
    # creating a csv writer object   
    csvwriter = csv.writer(csvfile)   
          
    # writing the fields   
    csvwriter.writerow(fields)   
          
    # writing the data rows   
    csvwriter.writerows(prediction) 